In [1]:
### HomeWork On Classification

In [4]:
!uv add scikit-learn


Resolved 51 packages in 1ms
Audited 19 packages in 0.13ms


In [5]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.metrics import mutual_info_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression


In [6]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

--2025-10-15 20:08:17--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv’

course_lead_scoring 100%[===================>]  78.98K  5.26KB/s    in 22s     

2025-10-15 20:08:55 (3.61 KB/s) - ‘course_lead_scoring.csv’ saved [80876/80876]



In [8]:
df = pd.read_csv("course_lead_scoring.csv")
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [15]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [16]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [31]:
numerical = df.dtypes[df.dtypes != "object"].index.to_list()
categorical = df.dtypes[df.dtypes == "object"].index.to_list()

df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [33]:
df[categorical] = df[categorical].fillna("NA")
df[numerical] = df[numerical].fillna(0.0)

In [45]:
numerical.remove("converted")

In [35]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [44]:
for i in df.industry.unique():
    print(f"{i} {df.industry[df.industry == i].count()}")

NA 134
retail 203
healthcare 187
education 187
manufacturing 174
technology 179
other 198
finance 200


In [46]:
df[numerical].corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


In [47]:
print("interaction_count and annual_income")

interaction_count and annual_income


In [49]:
train_test_split?

Signature:
train_test_split(
    *arrays,
    test_size=None,
    train_size=None,
    random_state=None,
    shuffle=True,
    stratify=None,
)
Docstring:
Split arrays or matrices into random train and test subsets.

Quick utility that wraps input validation,
``next(ShuffleSplit().split(X, y))``, and application to input data
into a single call for splitting (and optionally subsampling) data into a
one-liner.

Read more in the :ref:`User Guide <cross_validation>`.

Parameters
----------
*arrays : sequence of indexables with same length / shape[0]
    Allowed inputs are lists, numpy arrays, scipy-sparse
    matrices or pandas dataframes.

test_size : float or int, default=None
    If float, should be between 0.0 and 1.0 and represent the proportion
    of the dataset to include in the test split. If int, represents the
    absolute number of test samples. If None, the value is set to the
    complement of the train size. If ``train_size`` is also None, it will
    be set to 0.25.

trai

In [55]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [56]:
len(df_train), len(df_test), len(df_val)

(876, 293, 293)

In [ ]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [59]:
y_train = df_train['converted']
y_val = df_val['converted']
y_test = df_test['converted']

In [60]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

In [66]:
from IPython.display import display
for i in categorical:
    score = mutual_info_score(df_train[i], y_train)
    print(i, round(score, 2))
    
    

lead_source 0.04
industry 0.01
employment_status 0.01
location 0.0


In [69]:
df_train_dict = df_train.to_dict(orient="records")
df_val_dict = df_val.to_dict(orient="records")
df_test_dict = df_test.to_dict(orient="records")
df_train.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
1077,paid_ads,retail,0,58472.0,student,middle_east,5,0.03
463,organic_search,manufacturing,3,71738.0,student,middle_east,6,0.77
842,paid_ads,technology,3,81973.0,employed,north_america,2,0.59
835,NA,technology,1,74956.0,employed,europe,3,0.34
837,organic_search,retail,3,59335.0,student,australia,1,0.98


In [70]:
dv = DictVectorizer()
X_train = dv.fit_transform(df_train_dict)
X_val = dv.transform(df_val_dict)
X_test = dv.transform(df_test_dict)

In [71]:
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train) 

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [106]:
y_pred = model.predict_proba(X_val)[:, 1]
y_pred[:5]

array([0.61192163, 0.79982617, 0.53021344, 0.47131479, 0.57066132])

In [107]:
decision = (y_pred >= 0.5)

In [110]:
global_acc = (decision == y_val).mean()

In [90]:
global_weight = y_train.mean()
global_weight

np.float64(0.6244292237442922)

In [109]:
enumerate?

Init signature: enumerate(iterable, start=0)
Docstring:     
Return an enumerate object.

  iterable
    an object supporting iteration

The enumerate object yields pairs containing a count (from start, which
defaults to zero) and a value yielded by the iterable argument.

enumerate is useful for obtaining an indexed list:
    (0, seq[0]), (1, seq[1]), (2, seq[2]), ...
Type:           type
Subclasses:     

In [120]:
from copy import deepcopy

c = categorical + numerical
df_loop = pd.DataFrame(columns=["accuracy", "difference"])

print("Name\t acc\t difference\t")
for i in c:
    
    cols = deepcopy(c)
    cols.remove(i)
    
    dv = DictVectorizer(sparse=False)
    
    loop_train_dict = df_train[cols].to_dict(orient="records")
    loop_val_dict = df_val[cols].to_dict(orient="records")
    
    X_train = dv.fit_transform(loop_train_dict)
    X_val = dv.transform(loop_val_dict)
    
    model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
    
    model.fit(X_train, y_train)
    
    loop_pred_proba = model.predict_proba(X_val)
    loop_decision = (loop_pred_proba >= 0.5) [:,1]
    loop_acc = (loop_decision == y_val).mean()
    
    df_loop.loc[i]  = [loop_acc, global_acc - loop_acc]


Name	 acc	 difference	


In [130]:
df_loop['difference'].abs()

lead_source                 0.003413
industry                    0.000000
employment_status           0.003413
location                    0.010239
number_of_courses_viewed    0.143345
annual_income               0.153584
interaction_count           0.143345
lead_score                  0.006826
Name: difference, dtype: float64

In [140]:
C = [0.01, 0.1, 1, 10, 100]

loop_val_dict = df_val.to_dict(orient="records")
loop_train_dict = df_train.to_dict(orient="records")

dv = DictVectorizer(sparse=False)

X_train = dv.fit_transform(loop_train_dict)
X_val = dv.transform(loop_val_dict)

print("Name\t acc\t ")

for x in range(-10, 10):
    i = pow(10, x)
    model_loop = LogisticRegression(solver="liblinear", C=i
                               , max_iter=1000, random_state=42)
    
    model_loop.fit(X_train, y_train)
    
    loop_pred_proba = model_loop.predict_proba(X_val)
    loop_decision = (loop_pred_proba >= 0.5) [:,1]
    loop_acc = (loop_decision == y_val).mean()
    
    # print(f"{i}\t {round(loop_acc, 3)}\t {round(global_acc - loop_acc, 3)}\n")
    print(f"{i}\t {loop_acc}\t")


Name	 acc	 
1e-10	 0.5563139931740614	
1e-09	 0.5563139931740614	
1e-08	 0.5563139931740614	
1e-07	 0.5563139931740614	
1e-06	 0.5563139931740614	
1e-05	 0.5563139931740614	
0.0001	 0.5563139931740614	
0.001	 0.5631399317406144	
0.01	 0.6996587030716723	
0.1	 0.6996587030716723	
1	 0.6996587030716723	
10	 0.6996587030716723	
100	 0.6996587030716723	
1000	 0.6996587030716723	
10000	 0.6996587030716723	
100000	 0.6996587030716723	
1000000	 0.6996587030716723	
10000000	 0.6996587030716723	
100000000	 0.6996587030716723	
1000000000	 0.6996587030716723	
